# ETL

Importación de Librerías

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

Acceso al archivo de datos y asignación a un dataframe 

In [ ]:
# Acceso al archivo csv y asignación a un dataframe 
df_vfed = pd.read_csv('sources/Vehicle Fuel Economy Data.csv')
df = pd.read_csv('sources/costo_operacional_vehiculos.csv')

In [ ]:
# Acceso al archivo parquet y asignación a un dataframe 
df_yellow_tripdata = pd.read_parquet('sources\yellow_tripdata_2024-01.parquet')
df_green_tripdata = pd.read_parquet('sources\green_tripdata_2024-01.parquet')
df_fhv_tripdata = pd.read_parquet('sources\\fhv_tripdata_2024-01.parquet')

Visualización de la información del Dataframe

In [ ]:
df.info()
df_yellow_tripdata.info()
df_green_tripdata.info()
df_fhv_tripdata.info()
df_vfed.info()

Visualización de los Datos Iniciales del Dataframe

In [ ]:
df_vfed.head()
df.head()
df_yellow_tripdata.head()
df_green_tripdata.head()
df_fhv_tripdata.head()

Visualización de los Datos Finales del Dataframe

In [ ]:
df_vfed.tail()
df.tail()
df_yellow_tripdata.tail()
df_green_tripdata.tail()
df_fhv_tripdata.tail()

Mantener columnas necesarias en el Dataframe

Se recomienda usar el metodo drop ya que el dataframe original se va a usar modificandolo y no es necesario crear un dataframe nuevo que consume memoria y es menos eficiente.

In [ ]:
#ETL1
#  Eliminación de columnas no necesarias
df_vfed_col_eli = ['barrelsA08','charge240','fuelType','barrels08','range',
       'city08U','cityA08','cityA08U','cityCD','cityE','cityUF',
       'comb08', 'comb08U','combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF',
       'cylinders','displ', 'drive', 'engId', 'eng_dscr', 'feScore',
       'ghgScore', 'ghgScoreA','co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U',
       'highwayCD', 'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4',
       'mpgData', 'phevBlended', 'pv2', 'pv4','rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'youSaveSpend', 'guzzler', 'trans_dscr',
       'tCharger', 'sCharger', 'atvType', 'evMotor','rangeA',
       'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr', 'createdOn',
       'modifiedOn', 'startStop', 'phevCity', 'phevHwy', 'phevComb']
df_vfed.drop(df_vfed_col_eli, axis=1, inplace=True)
df_vfed = df_vfed.reset_index(drop=True)

In [ ]:
# ETL 2
# Eliminar las columnas innecesarias
columns_to_keep = ['Manuf', 'Model', 'Desc', 'Fuel_Type', 'Fuel_Cost', 'Electric_Cost', 'Total_Cost', 'Noise_Level']
df_clean = df[columns_to_keep].copy()

Renombrar Columnas

In [ ]:
# ETL 1
df_vfed.rename(columns={'city08':'Miles per gallon (mpg)','co2':'CO2 (p/mile)','fuelCost08':'FuelCost',
                        'fuelCostA08':'FuelCostA','fuelType1':'Fuel',
                        'VClass':'Category','fuelType2':'Alternative Fuel'}, inplace=True)

Modificar valores de las columnas

In [ ]:
# ETL 1
# Cambiamos NaN del Combustible Alternativo a No
df_vfed['Alternative Fuel'] = df_vfed['Alternative Fuel'].fillna('No')

In [ ]:
# ETL 2
# Eliminar el símbolo de libra y convertir las columnas de costos a numéricas
df_clean['Fuel_Cost'] = df_clean['Fuel_Cost'].replace('[£,]', '', regex=True).astype(float)
df_clean['Electric_Cost'] = df_clean['Electric_Cost'].replace('[£,]', '', regex=True).astype(float)
df_clean['Total_Cost'] = df_clean['Total_Cost'].replace('[£,]', '', regex=True).astype(float)

# Asumamos una tasa de conversión de 1 libra esterlina = 1.17 euros y 1 euro = 1.10 dólares
gbp_to_eur = 1.17
eur_to_usd = 1.10

# Convertir los costos de libras a euros y luego a dólares
df_clean['Fuel_Cost'] = df_clean['Fuel_Cost'] * gbp_to_eur * eur_to_usd
df_clean['Electric_Cost'] = df_clean['Electric_Cost'] * gbp_to_eur * eur_to_usd
df_clean['Total_Cost'] = df_clean['Total_Cost'] * gbp_to_eur * eur_to_usd

# Factor de conversión de millas a kilómetros
miles_to_km = 1.60934

# Ajustar los costos de 10,000 millas a 10,000 kilómetros
conversion_factor = 10000 / (10000 * miles_to_km)

df_clean['Fuel_Cost'] *= conversion_factor
df_clean['Electric_Cost'] *= conversion_factor
df_clean['Total_Cost'] *= conversion_factor

In [ ]:
# ETL 3
# Transformaciones adicionales (ejemplo: convertir formatos de fecha)
if 'pickup_datetime' in df_fhv_tripdata.columns:
    df_fhv_tripdata['pickup_datetime'] = pd.to_datetime(df_fhv_tripdata['pickup_datetime'])
if 'dropoff_datetime' in df.columns:
    df_fhv_tripdata['dropoff_datetime'] = pd.to_datetime(df_fhv_tripdata['dropoff_datetime'])

Modificar filas

In [ ]:
#ETL 1
# Eliminamos las filas donde 'Fuel' es nulo
df_vfed = df_vfed[df_vfed['Fuel'].notna()]

Eliminar Duplicados

In [ ]:
#ETL 1
# Eliminamos las filas con duplicados
df_vfed = df_vfed.drop_duplicates()
df_vfed.reset_index(drop=True, inplace=True)

In [ ]:
#ETL 2
# Eliminar duplicados (mantener la última ocurrencia)
df_clean = df_clean.drop_duplicates()
df_clean.reset_index(drop=True, inplace=True)

In [ ]:
# ETL 3
# Limpiar datos: eliminar duplicados
df_yellow_tripdata = df_yellow_tripdata.drop_duplicates()
df_yellow_tripdata.reset_index(drop=True, inplace=True)
df_green_tripdata = df_green_tripdata.drop_duplicates()
df_green_tripdata.reset_index(drop=True, inplace=True)
df_fhv_tripdata = df_green_tripdata.drop_duplicates()
df_fhv_tripdata.reset_index(drop=True, inplace=True)

Guardar el Dataframe en la carpeta Data

In [ ]:
#ETL 1
# Guarda el dataframe en la carpeta Data
df_vfed.to_parquet('Data/df_vfed.parquet', index=False)

In [ ]:
#ETL 2
# Guarda el dataframe en la carpeta Data
df_clean.to_csv('Data/costo_operacional_vehiculos_clean.csv', index=False)

In [ ]:
#ETL 3
# Guarda el dataframe en la carpeta Data
df_yellow_tripdata.to_parquet('Data/cleaned_yellow_tripdata_2024-01.parquet', index=False)
df_green_tripdata.to_parquet('Data/cleaned_green_tripdata_2024-01.parquet', index=False)
df_fhv_tripdata.to_parquet('Data/cleaned_fhv_tripdata_2024-01.parquet', index=False)

Por una cuestion de tamaño que GitHub recomienda no subir archivos que pesen mas de 50mb, dividimos el archivo en 5 partes para luego en el EDA unirlos y seguir trabajando sin problemas

In [ ]:
#ETL 3
import numpy as np

# Leer el archivo parquet
df = pd.read_parquet('Data/cleaned_yellow_tripdata_2024-01.parquet')

# Número de partes en las que quieres dividir el DataFrame
num_parts = 5  # Cambia esto según el número de partes que desees

# Dividir el DataFrame en partes iguales
chunks = np.array_split(df, num_parts)

# Guardar cada parte en un archivo parquet separado
for i, chunk in enumerate(chunks):
    chunk.to_parquet(f'Data/cleaned_yellow_tripdata_part_{i+1}.parquet', index=False)

Verificación de Archivos Generados

In [ ]:
#ETL 3
import os

# Verifica los archivos generados
for filename in os.listdir('Data'):
    if filename.startswith('cleaned_yellow_tripdata_part_') and filename.endswith('.parquet'):
        print(filename)

Eliminación de Archivo

In [ ]:
#ETL 3
# Eliminar el archivo
if os.path.exists('Data/cleaned_yellow_tripdata_2024-01.parquet'):
    os.remove('Data/cleaned_yellow_tripdata_2024-01.parquet')
    print(f'Archivo "Data/cleaned_yellow_tripdata_2024-01.parquet" eliminado exitosamente.')
else:
    print(f'El archivo "Data/cleaned_yellow_tripdata_2024-01.parquet" no existe.')